# 论文19：咖啡自动机 - 深入探讨不可逆性
**论文**: Scott Aaronson (2016) - "咖啡自动机"

**核心问题**: 为什么不能把牛奶和咖啡分开？不可逆性告诉了我们关于计算、信息和时间本身性质的什么？

---

## 不可逆性之谜

将牛奶滴入咖啡中。看着它扩散、旋转、混合直到均匀。现在尝试逆转它。**你不能。**

但这里有个谜题：
- 牛顿定律是**时间可逆的**：向后运行定律完全有效
- 每个分子碰撞是可逆的
- 微观定律不偏向任何时间方向

那么**不可逆性来自哪里**？

这不仅仅是物理学——它深刻地联系着：
- **计算**：我们可以逆转计算吗？
- **信息**："遗忘"真正意味着什么？
- **机器学习**：为什么神经网络会压缩信息？
- **时间箭头**：为什么时间有方向？

---

## 本实现

我们将从多个角度探讨不可逆性：

1. **咖啡混合**：扩散和熵增长
2. **相空间**：可逆性存在的地方
3. **粗粒化**：我们如何丢失信息
4. **庞加莱回归**：宇宙*将*取消混合
5. **麦克斯韦妖**：智能能逆转熵吗？
6. **兰道尔原理**：信息擦除的能量成本
7. **计算不可逆性**：单向函数和哈希
8. **机器学习**：为什么神经网络压缩和遗忘
9. **第二定律**：统计力学视角
10. **现代见解**：计算的热力学

这远超论文 1 对复杂度的介绍。我们将深入探讨**为什么**会出现不可逆性以及它对计算和学习的意义。

让我们开始吧！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.ndimage import convolve
from scipy.stats import entropy as scipy_entropy
from typing import Tuple, List, Callable
import hashlib
from dataclasses import dataclass

np.random.seed(42)

# 设置中文字体
plt.rcParams["font.family"] = ["Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully!")
print("NumPy version:", np.__version__)
print("\nReady to explore irreversibility...")

# 第1节：咖啡混合 - 扩散过程

从经典例子开始：牛奶在咖啡中扩散。

## 扩散方程

浓度 $c(x, y, t)$ 演化遵循：

$$\frac{\partial c}{\partial t} = D \nabla^2 c = D \left(\frac{\partial^2 c}{\partial x^2} + \frac{\partial^2 c}{\partial y^2}\right)$$

其中 $D$ 是**扩散系数**。

**关键洞察**：这个方程有**时间箭头**。向后运行看起来不同：
- 向前：浓度扩散（牛奶混合）
- 向后：浓度会聚集（牛奶取消混合）

但第二种情景违反热力学第二定律！

In [ ]:
def initialize_coffee_cup(size: int = 64) -> np.ndarray:
    """Initialize a 'coffee cup' with a drop of milk in the center.
    
    Returns:
        concentration: 2D array where 1 = milk, 0 = coffee
    """
    cup = np.zeros((size, size))
    
    # Add a drop of milk in the center
    center = size // 2
    radius = size // 8
    
    y, x = np.ogrid[:size, :size]
    mask = (x - center)**2 + (y - center)**2 <= radius**2
    cup[mask] = 1.0
    
    return cup


def diffusion_step(concentration: np.ndarray, D: float = 0.1) -> np.ndarray:
    """One timestep of diffusion using finite differences.
    
    Args:
        concentration: Current concentration field
        D: Diffusion coefficient
    
    Returns:
        New concentration after one timestep
    """
    # Laplacian kernel (discrete ∇²)
    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])
    
    # Apply Laplacian
    laplacian = convolve(concentration, kernel, mode='constant', cval=0.0)
    
    # Update: c(t+Δt) = c(t) + D·Δt·∇²c
    dt = 0.1
    new_concentration = concentration + D * dt * laplacian
    
    # Keep concentrations in valid range
    new_concentration = np.clip(new_concentration, 0, 1)
    
    return new_concentration


def simulate_coffee_mixing(steps: int = 200, D: float = 0.1) -> List[np.ndarray]:
    """Simulate coffee mixing over time.
    
    Returns:
        List of concentration fields at each timestep
    """
    cup = initialize_coffee_cup()
    history = [cup.copy()]
    
    for _ in range(steps):
        cup = diffusion_step(cup, D)
        history.append(cup.copy())
    
    return history


# Simulate mixing
print("Simulating coffee mixing...\n")
mixing_history = simulate_coffee_mixing(steps=200)

# Show key frames
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
timesteps = [0, 25, 50, 100, 200]

for ax, t in zip(axes, timesteps):
    im = ax.imshow(mixing_history[t], cmap='RdYlBu_r', vmin=0, vmax=1)
    ax.set_title(f't = {t}')
    ax.axis('off')

plt.colorbar(im, ax=axes, label='Milk Concentration', fraction=0.046)
plt.suptitle('Coffee Mixing: Irreversible Diffusion', fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("Observation: The milk spreads out and can never spontaneously unmix!")
print("\n✓ Diffusion simulation complete!")

# 第2节：熵增长 - 量化不可逆性

**熵**测量无序或"扩散程度"。随着牛奶混合，熵增加。

## 香农熵

将杯子分成若干箱。香农熵为：

$$H = -\sum_i p_i \log_2 p_i$$

其中 $p_i$ 是在箱子 $i$ 中找到牛奶的概率。

## 热力学熵

与与宏观态一致的微观态数量 $\Omega$ 相关：

$$S = k_B \ln \Omega$$

**热力学第二定律**：在孤立系统中，熵永不减少：

$$\frac{dS}{dt} \geq 0$$

这就是时间箭头！

In [ ]:
def compute_shannon_entropy(concentration: np.ndarray, num_bins: int = 10) -> float:
    """Compute Shannon entropy of concentration distribution.
    
    Args:
        concentration: 2D concentration field
        num_bins: Number of bins for histogram
    
    Returns:
        Shannon entropy in bits
    """
    # Flatten and create histogram
    flat = concentration.flatten()
    hist, _ = np.histogram(flat, bins=num_bins, range=(0, 1), density=True)
    
    # Normalize to probabilities
    hist = hist / hist.sum()
    
    # Compute Shannon entropy
    return scipy_entropy(hist, base=2)


def compute_spatial_entropy(concentration: np.ndarray) -> float:
    """Compute spatial entropy (variance of concentration).
    
    As mixing proceeds, variance decreases (becomes more uniform).
    So we use negative variance as a measure of mixing.
    """
    return -np.var(concentration)


def compute_mixing_quality(concentration: np.ndarray) -> float:
    """Compute how well-mixed the cup is.
    
    Returns:
        Value in [0, 1] where 1 = perfectly mixed
    """
    # Perfect mixing = all pixels have same concentration
    mean_concentration = concentration.mean()
    variance = np.var(concentration)
    
    # Maximum variance (for this mean) is when half is 0, half is 2*mean
    max_variance = mean_concentration * (1 - mean_concentration)
    
    if max_variance == 0:
        return 1.0
    
    return 1 - (variance / max_variance)


# Compute entropy over time
print("Computing entropy over time...\n")

shannon_entropies = [compute_shannon_entropy(cup) for cup in mixing_history]
spatial_entropies = [compute_spatial_entropy(cup) for cup in mixing_history]
mixing_qualities = [compute_mixing_quality(cup) for cup in mixing_history]

# Plot entropy growth
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Shannon entropy
axes[0].plot(shannon_entropies, linewidth=2, color='darkblue')
axes[0].set_xlabel('Time Step', fontsize=11)
axes[0].set_ylabel('Shannon Entropy (bits)', fontsize=11)
axes[0].set_title('Information Entropy Growth', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=shannon_entropies[0], color='red', linestyle='--', alpha=0.5, label='Initial')
axes[0].legend()

# Plot 2: Spatial entropy (negative variance)
axes[1].plot(spatial_entropies, linewidth=2, color='darkgreen')
axes[1].set_xlabel('Time Step', fontsize=11)
axes[1].set_ylabel('Spatial Entropy (-Variance)', fontsize=11)
axes[1].set_title('Spatial Disorder Growth', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Plot 3: Mixing quality
axes[2].plot(mixing_qualities, linewidth=2, color='darkorange')
axes[2].set_xlabel('Time Step', fontsize=11)
axes[2].set_ylabel('Mixing Quality', fontsize=11)
axes[2].set_title('Approach to Equilibrium', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Perfect mixing')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"Initial Shannon entropy: {shannon_entropies[0]:.3f} bits")
print(f"Final Shannon entropy:   {shannon_entropies[-1]:.3f} bits")
print(f"Entropy increase:        {shannon_entropies[-1] - shannon_entropies[0]:.3f} bits")
print(f"\nFinal mixing quality:    {mixing_qualities[-1]:.1%}")

print("\n🔑 Key insight: Entropy monotonically increases!")
print("   This is the second law of thermodynamics in action.")
print("\n✓ Entropy analysis complete!")

# 第3节：相空间与刘维尔定理

这里变得深刻：**为什么熵增加如果微观定律是可逆的？**

## 相空间

考虑一个有 $N$ 个粒子的系统。**相空间**是所有位置和动量的 6N 维空间：

$$\Gamma = (x_1, y_1, z_1, p_{x1}, p_{y1}, p_{z1}, \ldots, x_N, y_N, z_N, p_{xN}, p_{yN}, p_{zN})$$

相空间中的一点是一个**微观态**——系统的完整规范。

## 刘维尔定理

**刘维尔定理**指出相空间体积守恒：

$$\frac{d}{dt} \int_{V} d\Gamma = 0$$

这意味着：
- **微观上，动力学是可逆的**
- 相空间体积不改变
- 信息守恒

## 解决方案：粗粒化

关键是**粗粒化**：我们无法跟踪单个分子，所以将附近的微观态分组为**宏观态**。

- **微观态**：所有分子的精确位置/动量（可逆）
- **宏观态**：粗略描述，如"温度"或"浓度"（不可逆）

**熵增加**是因为：
1. 许多微观态映射到同一宏观态
2. 演化趋向于具有更多微观态的宏观态
3. 粗粒化时我们丢失信息

让我们模拟一下！

In [ ]:
@dataclass
class Particle:
    """A particle in 2D phase space."""
    x: float
    y: float
    vx: float
    vy: float


def initialize_particles(num_particles: int = 100, region: str = 'left') -> List[Particle]:
    """Initialize particles in a specific region.
    
    Args:
        num_particles: Number of particles
        region: 'left' or 'right' half of box
    """
    particles = []
    
    for _ in range(num_particles):
        if region == 'left':
            x = np.random.uniform(0.1, 0.4)
        else:
            x = np.random.uniform(0.6, 0.9)
        
        y = np.random.uniform(0.1, 0.9)
        
        # Random velocities
        speed = 0.02
        angle = np.random.uniform(0, 2*np.pi)
        vx = speed * np.cos(angle)
        vy = speed * np.sin(angle)
        
        particles.append(Particle(x, y, vx, vy))
    
    return particles


def update_particles(particles: List[Particle], dt: float = 1.0) -> List[Particle]:
    """Update particle positions (free motion with reflecting boundaries)."""
    new_particles = []
    
    for p in particles:
        # Update position
        new_x = p.x + p.vx * dt
        new_y = p.y + p.vy * dt
        new_vx, new_vy = p.vx, p.vy
        
        # Reflecting boundaries
        if new_x < 0 or new_x > 1:
            new_vx = -new_vx
            new_x = np.clip(new_x, 0, 1)
        
        if new_y < 0 or new_y > 1:
            new_vy = -new_vy
            new_y = np.clip(new_y, 0, 1)
        
        new_particles.append(Particle(new_x, new_y, new_vx, new_vy))
    
    return new_particles


def compute_macrostate(particles: List[Particle], num_bins: int = 4) -> np.ndarray:
    """Coarse-grain particles into spatial bins.
    
    Returns:
        2D histogram of particle counts
    """
    positions = np.array([[p.x, p.y] for p in particles])
    
    hist, _, _ = np.histogram2d(
        positions[:, 0], positions[:, 1],
        bins=num_bins,
        range=[[0, 1], [0, 1]]
    )
    
    return hist


def compute_macrostate_entropy(macrostate: np.ndarray) -> float:
    """Compute entropy of a macrostate."""
    # Flatten and normalize
    counts = macrostate.flatten()
    if counts.sum() == 0:
        return 0.0
    
    probs = counts / counts.sum()
    probs = probs[probs > 0]  # Remove zeros
    
    return -np.sum(probs * np.log2(probs))


# Simulate particle mixing
print("Simulating particle mixing with coarse-graining...\n")

# Initialize particles on left side
particles = initialize_particles(num_particles=200, region='left')

# Simulate
num_steps = 500
particle_history = [particles]
macrostate_history = []
macrostate_entropies = []

for step in range(num_steps):
    particles = update_particles(particles)
    particle_history.append([Particle(p.x, p.y, p.vx, p.vy) for p in particles])
    
    # Compute macrostate every 10 steps
    if step % 10 == 0:
        macrostate = compute_macrostate(particles)
        macrostate_history.append(macrostate)
        macrostate_entropies.append(compute_macrostate_entropy(macrostate))

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Top row: Microstate (particle positions)
timesteps_vis = [0, 100, 500]
for idx, t in enumerate(timesteps_vis):
    ax = axes[0, idx]
    ps = particle_history[t]
    positions = np.array([[p.x, p.y] for p in ps])
    
    ax.scatter(positions[:, 0], positions[:, 1], s=10, alpha=0.6, color='blue')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_title(f'Microstate at t={t}', fontweight='bold')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

# Bottom row: Macrostate (coarse-grained)
macro_timesteps = [0, 10, 50]
for idx, mt in enumerate(macro_timesteps):
    ax = axes[1, idx]
    im = ax.imshow(macrostate_history[mt].T, cmap='Blues', origin='lower', 
                   extent=[0, 1, 0, 1], vmin=0, vmax=macrostate_history[0].max())
    ax.set_title(f'Macrostate at t={mt*10}\nS={macrostate_entropies[mt]:.2f} bits', 
                fontweight='bold')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.colorbar(im, ax=ax, label='Particle count')

plt.suptitle('Microscopic Reversibility vs Macroscopic Irreversibility', 
            fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# Plot entropy growth
plt.figure(figsize=(10, 5))
plt.plot(np.arange(len(macrostate_entropies)) * 10, macrostate_entropies, 
         linewidth=2, color='darkred')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Macroscopic Entropy (bits)', fontsize=12)
plt.title('Entropy Growth Despite Microscopic Reversibility', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("🔑 Key insight: Liouville's theorem guarantees microscopic reversibility.")
print("   But macroscopic entropy STILL increases due to coarse-graining!")
print("\n   • Microstate: Exact particle positions (reversible)")
print("   • Macrostate: Binned particle counts (irreversible)")
print("   • Many microstates → same macrostate")
print("   • Evolution explores more microstates over time")
print("\n✓ Phase space analysis complete!")

# 第4节：庞加莱回归 - 宇宙将会取消混合！

这里有一个令人震惊的事实：**庞加莱回归定理**指出，任何有限系统最终将任意接近其初始状态。

## 定理

对于**有限**相空间体积，几乎每个初始条件都会无限次地循环：

$$\lim_{t \to \infty} \inf \, d(\Gamma(t), \Gamma(0)) = 0$$

这意味着：
- 给定足够时间，咖啡**将**取消混合！
- 粒子**将**返回到其初始配置！

## 陷阱：回归时间

你需要等待多久？对于 $N$ 个粒子：

$$t_{\text{recurrence}} \sim e^{N}$$

对于一个有 $N \sim 10^{23}$ 个分子的咖啡杯：

$$t_{\text{recurrence}} \sim 10^{10^{23}} \text{ 秒}$$

这**远大于**宇宙年龄（$\sim 10^{17}$ 秒）。

**实际不可逆性**：系统理论上可逆，但实际上不可逆。

让我们用一个小系统演示一下！

In [ ]:
def simple_phase_space_system(num_states: int = 8, num_steps: int = 10000) -> Tuple[List[int], List[int]]:
    """Simulate a simple deterministic system on a finite phase space.
    
    Args:
        num_states: Size of phase space (small for demonstration)
        num_steps: Number of steps to simulate
    
    Returns:
        states: List of states visited
        recurrence_times: Time to return to initial state
    """
    # Define a deterministic evolution rule
    # Simple example: state_next = (a * state + b) mod num_states
    a, b = 3, 1  # Parameters chosen to give interesting dynamics
    
    initial_state = 0
    state = initial_state
    states = [state]
    recurrence_times = []
    
    for step in range(1, num_steps):
        # Deterministic evolution
        state = (a * state + b) % num_states
        states.append(state)
        
        # Check for recurrence
        if state == initial_state:
            recurrence_times.append(step)
    
    return states, recurrence_times


def estimate_recurrence_time_scaling() -> Tuple[List[int], List[float]]:
    """Estimate how recurrence time scales with system size."""
    sizes = [4, 8, 16, 32, 64, 128]
    avg_recurrence_times = []
    
    for size in sizes:
        _, rec_times = simple_phase_space_system(num_states=size, num_steps=size*20)
        if len(rec_times) > 0:
            avg_time = np.mean(np.diff([0] + rec_times))
        else:
            avg_time = size * 10  # Estimate
        avg_recurrence_times.append(avg_time)
    
    return sizes, avg_recurrence_times


# Demonstrate Poincaré recurrence
print("Demonstrating Poincaré recurrence...\n")

num_states = 16
states, recurrence_times = simple_phase_space_system(num_states=num_states, num_steps=200)

print(f"System with {num_states} states")
print(f"Initial state: {states[0]}")
print(f"\nRecurrences found at timesteps: {recurrence_times[:5]}...")

if len(recurrence_times) > 1:
    period = recurrence_times[1] - recurrence_times[0]
    print(f"Recurrence period: {period} steps")

# Visualize state evolution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: State trajectory
ax = axes[0, 0]
ax.plot(states[:200], linewidth=1.5, color='darkblue')
for rt in recurrence_times:
    if rt < 200:
        ax.axvline(rt, color='red', alpha=0.3, linestyle='--')
ax.set_xlabel('Time Step', fontsize=11)
ax.set_ylabel('State', fontsize=11)
ax.set_title(f'State Evolution (Red lines = recurrence)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 2: State histogram
ax = axes[0, 1]
ax.hist(states, bins=num_states, edgecolor='black', alpha=0.7, color='steelblue')
ax.set_xlabel('State', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title('State Distribution (Should be uniform)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Plot 3: Recurrence time scaling
ax = axes[1, 0]
sizes, rec_times_avg = estimate_recurrence_time_scaling()
ax.semilogy(sizes, rec_times_avg, 'o-', linewidth=2, markersize=8, color='darkgreen', label='Observed')
# Fit exponential
log_rec_times = np.log(rec_times_avg)
coeffs = np.polyfit(sizes, log_rec_times, 1)
fit_line = np.exp(coeffs[1]) * np.exp(coeffs[0] * np.array(sizes))
ax.semilogy(sizes, fit_line, '--', linewidth=2, color='red', label=f'Exponential fit')
ax.set_xlabel('System Size (# states)', fontsize=11)
ax.set_ylabel('Average Recurrence Time', fontsize=11)
ax.set_title('Exponential Growth of Recurrence Time', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Extrapolation to realistic systems
ax = axes[1, 1]
N_values = np.array([10, 100, 1000, 10000, 1e23])  # Up to coffee cup
t_rec_estimates = np.exp(N_values * 0.5)  # Very rough estimate
universe_age = 4.3e17  # seconds

ax.loglog(N_values[:-1], t_rec_estimates[:-1], 'o-', linewidth=2, markersize=8, color='purple')
ax.axhline(universe_age, color='red', linestyle='--', linewidth=2, label='Age of universe')
ax.set_xlabel('Number of Particles', fontsize=11)
ax.set_ylabel('Recurrence Time (seconds)', fontsize=11)
ax.set_title('Why Coffee Never Unmixes', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, which='both')
ax.text(100, 1e20, f'Coffee cup:\n$N \\sim 10^{{23}}$\n$t_{{rec}} \\sim 10^{{10^{{23}}}}$ s', 
        fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n🔑 Key insights:")
print("   1. Poincaré recurrence IS real - finite systems eventually recur")
print("   2. But recurrence time grows EXPONENTIALLY with system size")
print(f"   3. For coffee (~10²³ particles): t_rec ~ 10^(10^23) seconds")
print(f"   4. Universe age: ~10^17 seconds")
print("   5. So coffee will unmix... after waiting 10^(10^23) times the age of the universe!")
print("\n   This is why irreversibility is PRACTICAL even though dynamics are reversible.")
print("\n✓ Poincaré recurrence analysis complete!")

# 第5节：麦克斯韦妖 - 智能能逆转熵吗？

**麦克斯韦妖**是一个挑战第二定律的思想实验。

## 设置

1. 盒子分成两部分的盒子，中间有一扇门
2. 气体分子随机移动
3. 一个"妖"操作门：
   - 向右开的门：快速分子
   - 向左开的门：慢速分子
4. 结果：右边是热气，左边是冷气（熵减少！）

## 悖论

妖似乎**违反第二定律**，不做功就能减少熵。

## 解决方案：兰道尔原理

妖必须**测量**分子速度并**存储**此信息。最终，它必须**擦除**记忆以继续操作。

**兰道尔原理**（1961）：擦除一位信息至少需要耗散：

$$E_{\text{min}} = k_B T \ln 2$$

的能量作为热量。

这热量耗散**精确补偿**了妖减少的熵！

**深刻含义**：**信息是物理的**。计算有热力学成本。

让我们模拟麦克斯韦妖！

In [ ]:
@dataclass
class GasParticle:
    """A gas particle with position and velocity."""
    x: float
    y: float
    vx: float
    vy: float
    
    def speed(self) -> float:
        return np.sqrt(self.vx**2 + self.vy**2)


class MaxwellsDemon:
    """Simulates Maxwell's demon thought experiment."""
    
    def __init__(self, num_particles: int = 100):
        self.num_particles = num_particles
        self.particles = self._initialize_particles()
        self.door_position = 0.5  # Middle of box
        self.memory = []  # Demon's memory
        self.entropy_cost = 0.0  # Cost of erasure
    
    def _initialize_particles(self) -> List[GasParticle]:
        """Initialize particles with Maxwell-Boltzmann distribution."""
        particles = []
        for _ in range(self.num_particles):
            x = np.random.uniform(0.1, 0.9)
            y = np.random.uniform(0.1, 0.9)
            
            # Maxwell-Boltzmann velocity distribution
            speed = np.random.rayleigh(0.02)
            angle = np.random.uniform(0, 2*np.pi)
            vx = speed * np.cos(angle)
            vy = speed * np.sin(angle)
            
            particles.append(GasParticle(x, y, vx, vy))
        return particles
    
    def update_without_demon(self, dt: float = 1.0):
        """Update particles without demon (natural evolution)."""
        new_particles = []
        for p in self.particles:
            new_x = p.x + p.vx * dt
            new_y = p.y + p.vy * dt
            new_vx, new_vy = p.vx, p.vy
            
            # Reflecting boundaries
            if new_x < 0 or new_x > 1:
                new_vx = -new_vx
                new_x = np.clip(new_x, 0, 1)
            if new_y < 0 or new_y > 1:
                new_vy = -new_vy
                new_y = np.clip(new_y, 0, 1)
            
            new_particles.append(GasParticle(new_x, new_y, new_vx, new_vy))
        
        self.particles = new_particles
    
    def update_with_demon(self, dt: float = 1.0, threshold_speed: float = 0.025):
        """Update particles with demon operating the door."""
        new_particles = []
        
        for p in self.particles:
            new_x = p.x + p.vx * dt
            new_y = p.y + p.vy * dt
            new_vx, new_vy = p.vx, p.vy
            
            # Check if particle crosses middle partition
            crosses_door = (p.x < self.door_position <= new_x) or (p.x > self.door_position >= new_x)
            
            if crosses_door and 0.4 < new_y < 0.6:  # Door is in middle vertically
                # Demon measures speed and decides
                speed = p.speed()
                self.memory.append(speed)  # Store measurement (costs memory)
                
                # Demon's rule:
                # - Fast particles go right
                # - Slow particles go left
                going_right = new_x > p.x
                is_fast = speed > threshold_speed
                
                if (going_right and not is_fast) or (not going_right and is_fast):
                    # Reflect the particle (close door)
                    new_vx = -new_vx
                    new_x = p.x
            
            # Regular boundaries
            if new_x < 0 or new_x > 1:
                new_vx = -new_vx
                new_x = np.clip(new_x, 0, 1)
            if new_y < 0 or new_y > 1:
                new_vy = -new_vy
                new_y = np.clip(new_y, 0, 1)
            
            new_particles.append(GasParticle(new_x, new_y, new_vx, new_vy))
        
        self.particles = new_particles
        
        # Landauer erasure cost
        if len(self.memory) > 50:  # Erase old memories
            bits_erased = len(self.memory) - 50
            self.entropy_cost += bits_erased * np.log(2)  # k_B T ln(2) per bit
            self.memory = self.memory[-50:]
    
    def compute_temperature_difference(self) -> float:
        """Compute temperature difference between left and right chambers."""
        left_particles = [p for p in self.particles if p.x < self.door_position]
        right_particles = [p for p in self.particles if p.x >= self.door_position]
        
        if len(left_particles) == 0 or len(right_particles) == 0:
            return 0.0
        
        # Temperature ∝ average kinetic energy ∝ average speed²
        left_temp = np.mean([p.speed()**2 for p in left_particles])
        right_temp = np.mean([p.speed()**2 for p in right_particles])
        
        return right_temp - left_temp


# Simulate with and without demon
print("Simulating Maxwell's demon...\n")

# Without demon
system_no_demon = MaxwellsDemon(num_particles=150)
temp_diffs_no_demon = []

for _ in range(300):
    system_no_demon.update_without_demon()
    temp_diffs_no_demon.append(system_no_demon.compute_temperature_difference())

# With demon
system_with_demon = MaxwellsDemon(num_particles=150)
temp_diffs_with_demon = []
entropy_costs = []

for _ in range(300):
    system_with_demon.update_with_demon()
    temp_diffs_with_demon.append(system_with_demon.compute_temperature_difference())
    entropy_costs.append(system_with_demon.entropy_cost)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Temperature difference without demon
axes[0, 0].plot(temp_diffs_no_demon, linewidth=2, color='blue', alpha=0.7)
axes[0, 0].axhline(0, color='black', linestyle='--', alpha=0.3)
axes[0, 0].set_xlabel('Time Step', fontsize=11)
axes[0, 0].set_ylabel('Temp Diff (Right - Left)', fontsize=11)
axes[0, 0].set_title('Without Demon: No Temperature Gradient', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Temperature difference with demon
axes[0, 1].plot(temp_diffs_with_demon, linewidth=2, color='red', alpha=0.7)
axes[0, 1].axhline(0, color='black', linestyle='--', alpha=0.3)
axes[0, 1].set_xlabel('Time Step', fontsize=11)
axes[0, 1].set_ylabel('Temp Diff (Right - Left)', fontsize=11)
axes[0, 1].set_title('With Demon: Temperature Gradient Created!', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Particle distribution (final state without demon)
ax = axes[1, 0]
positions_no_demon = np.array([[p.x, p.y] for p in system_no_demon.particles])
speeds_no_demon = np.array([p.speed() for p in system_no_demon.particles])
scatter = ax.scatter(positions_no_demon[:, 0], positions_no_demon[:, 1], 
                    c=speeds_no_demon, s=20, cmap='coolwarm', alpha=0.6)
ax.axvline(0.5, color='black', linewidth=2, linestyle='--', label='Partition')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xlabel('x', fontsize=11)
ax.set_ylabel('y', fontsize=11)
ax.set_title('Final State Without Demon', fontsize=12, fontweight='bold')
ax.legend()
plt.colorbar(scatter, ax=ax, label='Speed')

# Plot 4: Particle distribution (final state with demon)
ax = axes[1, 1]
positions_with_demon = np.array([[p.x, p.y] for p in system_with_demon.particles])
speeds_with_demon = np.array([p.speed() for p in system_with_demon.particles])
scatter = ax.scatter(positions_with_demon[:, 0], positions_with_demon[:, 1],
                    c=speeds_with_demon, s=20, cmap='coolwarm', alpha=0.6)
ax.axvline(0.5, color='black', linewidth=2, linestyle='--', label='Partition')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xlabel('x', fontsize=11)
ax.set_ylabel('y', fontsize=11)
ax.set_title('Final State With Demon: Fast→Right, Slow→Left', fontsize=12, fontweight='bold')
ax.legend()
plt.colorbar(scatter, ax=ax, label='Speed')

plt.tight_layout()
plt.show()

# Landauer's principle
plt.figure(figsize=(10, 5))
plt.plot(entropy_costs, linewidth=2, color='darkgreen')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Cumulative Entropy Cost (k_B ln 2 per bit)', fontsize=12)
plt.title("Landauer's Principle: Information Erasure Has Entropic Cost", 
         fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔑 Key insights:")
print("   1. Demon CAN create temperature gradient (decrease entropy locally)")
print("   2. But demon must MEASURE and REMEMBER particle speeds")
print("   3. Memory is finite → must ERASE old measurements")
print(f"   4. Landauer: Erasing 1 bit releases ≥ k_B T ln(2) heat")
print(f"   5. Total entropy cost from erasure: {entropy_costs[-1]:.2f} k_B ln(2)")
print("   6. This EXACTLY compensates for entropy decrease!")
print("\n   → The second law is saved! Information is physical.")
print("\n✓ Maxwell's demon simulation complete!")

# 第6节：计算不可逆性 - 单向函数

不可逆性不仅仅是物理学——它是**计算**的核心！

## 单向函数

函数 $f$ 是**单向的**，如果：
1. 易于计算：$y = f(x)$ 很快
2. 难以求逆：给定 $y$，找到 $x$ 使得 $f(x) = y$ 很难

**示例**：
- **乘法**：$f(p, q) = p \times q$（容易）
- **因式分解**：给定 $n = p \times q$，找 $p, q$（对大 $n$ 很难）
- **加密哈希**：SHA-256 等

## 与热力学的联系

单向函数是**计算上不可逆的**：
- 计算时信息被**销毁**
- 多个输入 $x$ 映射到相同输出 $y$
- 这就像热力学中的**粗粒化**！

## 计算的兰道尔原理

**不可逆计算**（信息被销毁）有最小能量成本：

$$E_{\text{min}} = k_B T \ln(2) \times (\text{销毁的位数})$$

**可逆计算**（没有信息销毁）原则上可以**零能量成本**完成！

这就是为什么量子计算机被设计为可逆的。

In [ ]:
def hash_function(x: int, num_bits_out: int = 8) -> int:
    """Simple hash function (one-way).
    
    Args:
        x: Input integer
        num_bits_out: Number of bits in output
    
    Returns:
        Hash value in [0, 2^num_bits_out - 1]
    """
    # Use Python's built-in hash with modulo
    return hash(x) % (2 ** num_bits_out)


def cryptographic_hash(data: str) -> str:
    """Cryptographic hash using SHA-256."""
    return hashlib.sha256(data.encode()).hexdigest()


def demonstrate_collision(num_inputs: int = 1000, num_bits_out: int = 8) -> dict:
    """Demonstrate hash collisions (many inputs → same output)."""
    hash_map = {}
    
    for x in range(num_inputs):
        h = hash_function(x, num_bits_out)
        if h not in hash_map:
            hash_map[h] = []
        hash_map[h].append(x)
    
    return hash_map


def compute_information_loss(input_bits: int, output_bits: int) -> float:
    """Compute information loss in bits.
    
    When hashing n-bit input to m-bit output (m < n),
    we lose (n - m) bits of information.
    """
    return max(0, input_bits - output_bits)


# Demonstrate one-way functions
print("Demonstrating computational irreversibility...\n")

# Example 1: Simple hash collisions
print("Example 1: Hash Collisions")
print("="*50)
num_inputs = 1000
num_bits_out = 6  # Only 64 possible outputs
hash_map = demonstrate_collision(num_inputs, num_bits_out)

print(f"Hashed {num_inputs} inputs to {2**num_bits_out} possible outputs")
print(f"Number of collisions: {sum(1 for v in hash_map.values() if len(v) > 1)}")
print(f"Average inputs per output: {num_inputs / len(hash_map):.1f}")

# Show some collisions
collision_example = [v for v in hash_map.values() if len(v) > 2][0]
print(f"\nExample collision: Inputs {collision_example[:5]} all hash to {hash_function(collision_example[0], num_bits_out)}")

# Example 2: Cryptographic hash
print("\n" + "="*50)
print("Example 2: Cryptographic Hash (SHA-256)")
print("="*50)

messages = [
    "Hello, World!",
    "Hello, World.",  # One character different
    "The coffee automaton demonstrates irreversibility"
]

for msg in messages:
    hash_val = cryptographic_hash(msg)
    print(f"Message: '{msg[:40]}'")
    print(f"SHA-256: {hash_val[:32]}...")
    print()

print("Note: Tiny change in input → completely different hash (avalanche effect)")
print("This makes inversion computationally infeasible!")

# Visualize hash distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Hash collision distribution
ax = axes[0]
collision_counts = [len(v) for v in hash_map.values()]
ax.hist(collision_counts, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
ax.set_xlabel('Number of Inputs per Hash Value', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title('Hash Collisions: Many-to-One Mapping', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.axvline(np.mean(collision_counts), color='red', linestyle='--', linewidth=2,
          label=f'Mean: {np.mean(collision_counts):.1f}')
ax.legend()

# Plot 2: Information loss
ax = axes[1]
input_bits_range = np.arange(8, 65, 4)
output_bits_fixed = 16
info_loss = [compute_information_loss(ib, output_bits_fixed) for ib in input_bits_range]

ax.plot(input_bits_range, info_loss, 'o-', linewidth=2, markersize=8, color='darkred')
ax.fill_between(input_bits_range, 0, info_loss, alpha=0.3, color='red')
ax.set_xlabel('Input Size (bits)', fontsize=11)
ax.set_ylabel('Information Lost (bits)', fontsize=11)
ax.set_title(f'Information Loss in Hashing (→{output_bits_fixed} bits)', 
            fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Landauer's cost
print("\n" + "="*50)
print("Landauer's Principle for Computation")
print("="*50)

bits_destroyed = 48  # Hash 64-bit input to 16-bit output
k_B = 1.380649e-23  # Boltzmann constant (J/K)
T = 300  # Temperature (K)
energy_per_bit = k_B * T * np.log(2)  # Joules
total_energy = bits_destroyed * energy_per_bit

print(f"Hashing 64-bit input → 16-bit output:")
print(f"  Bits destroyed: {bits_destroyed}")
print(f"  Minimum energy cost: {total_energy:.2e} J")
print(f"  At T=300K: {energy_per_bit:.2e} J per bit")
print(f"\nModern CPUs use ~10^-9 J per operation (well above Landauer limit)")
print(f"Landauer limit: {energy_per_bit:.2e} J per bit erased")
print(f"Gap: ~10^6× above minimum! Room for improvement.")

print("\n🔑 Key insights:")
print("   1. One-way functions destroy information (irreversible)")
print("   2. Many inputs map to same output (collisions)")
print("   3. This is computational coarse-graining!")
print("   4. Landauer: Destroying 1 bit costs ≥ k_B T ln(2) energy")
print("   5. Reversible computation (no info loss) could be free!")
print("\n✓ Computational irreversibility demonstration complete!")

# 第7节：机器学习与信息瓶颈

**不可逆性对机器学习至关重要！**

## 为什么神经网络必须遗忘

神经网络是一个**有损压缩**函数：

$$f: \mathbb{R}^{d_{\text{in}}} \to \mathbb{R}^{d_{\text{out}}}$$

通常 $d_{\text{out}} \ll d_{\text{in}}$。

**信息瓶颈**：隐藏层压缩输入，丢弃无关细节：
- 输入：高维数据（例如图像像素）
- 隐藏层：渐进压缩
- 输出：低维表示（例如类别标签）

## 为什么压缩有帮助

1. **泛化**：遗忘噪声，记住信号
2. **效率**：只存储相关特征
3. **鲁棒性**：相似输入 → 相似输出

**与热力学的联系**：
- **不可逆压缩** = 丢失信息
- **熵增加** = 扩散不确定性
- **粗粒化** = 将相似输入分组

## 信息瓶颈原理

找到表示 $T$，使得：
$$\min_{T} [I(X; T) - \beta \cdot I(T; Y)]$$

- $I(X; T)$：$T$ 关于输入 $X$ 记住的信息（压缩）
- $I(T; Y)$：$T$ 关于输出 $Y$ 保留的信息（预测）
- $\beta$：权衡参数

**目标**：最大化压缩，同时保留预测能力。

让我们演示一下！

In [ ]:
def create_autoencoder_layers(input_dim: int, hidden_dims: List[int]) -> List[Tuple[np.ndarray, np.ndarray]]:
    """Create a simple autoencoder (compression then decompression).
    
    Returns:
        List of (W, b) for each layer
    """
    layers = []
    dims = [input_dim] + hidden_dims + [input_dim]
    
    for i in range(len(dims) - 1):
        W = np.random.randn(dims[i], dims[i+1]) * np.sqrt(2.0 / dims[i])
        b = np.zeros(dims[i+1])
        layers.append((W, b))
    
    return layers


def forward_autoencoder(x: np.ndarray, layers: List[Tuple]) -> Tuple[np.ndarray, List[np.ndarray]]:
    """Forward pass through autoencoder.
    
    Returns:
        reconstruction, list of activations at each layer
    """
    activations = [x]
    current = x
    
    for i, (W, b) in enumerate(layers):
        current = current @ W + b
        # ReLU for hidden layers, linear for output
        if i < len(layers) - 1:
            current = np.maximum(0, current)
        activations.append(current)
    
    return current, activations


def measure_information_content(activations: np.ndarray, num_bins: int = 20) -> float:
    """Estimate information content via entropy of activation distribution."""
    # Flatten activations
    flat = activations.flatten()
    
    # Create histogram
    hist, _ = np.histogram(flat, bins=num_bins, density=True)
    hist = hist / hist.sum()
    hist = hist[hist > 0]
    
    # Shannon entropy
    return -np.sum(hist * np.log2(hist))


# Generate synthetic data
print("Demonstrating information bottleneck in neural networks...\n")

# Create high-dimensional input with low-dimensional structure
num_samples = 500
input_dim = 50
latent_dim = 3  # True underlying dimensionality

# Generate data: low-dim latent → high-dim observation
latent = np.random.randn(num_samples, latent_dim)
projection = np.random.randn(latent_dim, input_dim)
X = latent @ projection
X += np.random.randn(num_samples, input_dim) * 0.5  # Add noise

print(f"Generated data:")
print(f"  Samples: {num_samples}")
print(f"  Input dimension: {input_dim}")
print(f"  True latent dimension: {latent_dim}")
print(f"  Information must be compressed {input_dim/latent_dim:.1f}×!\n")

# Create autoencoder with bottleneck
hidden_dims = [25, 10, 5]  # Progressively compress
layers = create_autoencoder_layers(input_dim, hidden_dims)

print(f"Autoencoder architecture: {input_dim} → {' → '.join(map(str, hidden_dims))} → {input_dim}")
print(f"Bottleneck: {hidden_dims[-1]} dimensions (compression: {input_dim/hidden_dims[-1]:.1f}×)\n")

# Analyze information flow
reconstructions = []
all_activations = []

for x in X[:100]:  # Use subset for speed
    recon, acts = forward_autoencoder(x, layers)
    reconstructions.append(recon)
    all_activations.append(acts)

# Measure information at each layer
layer_entropies = []
layer_sizes = [input_dim] + hidden_dims + [input_dim]

for layer_idx in range(len(layers) + 1):
    layer_activations = np.array([acts[layer_idx] for acts in all_activations])
    entropy = measure_information_content(layer_activations)
    layer_entropies.append(entropy)

# Visualize information bottleneck
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Network architecture
ax = axes[0, 0]
x_pos = np.arange(len(layer_sizes))
ax.bar(x_pos, layer_sizes, color=['blue' if s > hidden_dims[-1] else 'red' for s in layer_sizes],
       alpha=0.7, edgecolor='black')
ax.set_xticks(x_pos)
ax.set_xticklabels(['Input'] + [f'H{i+1}' for i in range(len(hidden_dims))] + ['Output'])
ax.set_ylabel('Layer Dimension', fontsize=11)
ax.set_title('Network Architecture: Information Bottleneck', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Plot 2: Information content per layer
ax = axes[0, 1]
ax.plot(x_pos, layer_entropies, 'o-', linewidth=2, markersize=10, color='darkgreen')
ax.fill_between(x_pos, 0, layer_entropies, alpha=0.3, color='green')
ax.set_xticks(x_pos)
ax.set_xticklabels(['Input'] + [f'H{i+1}' for i in range(len(hidden_dims))] + ['Output'])
ax.set_ylabel('Information Content (bits)', fontsize=11)
ax.set_title('Information Loss Through Network', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axvline(len(hidden_dims), color='red', linestyle='--', linewidth=2, label='Bottleneck')
ax.legend()

# Plot 3: Compression ratio
ax = axes[1, 0]
compression_ratios = [layer_sizes[0] / s for s in layer_sizes]
ax.bar(x_pos, compression_ratios, color='orange', alpha=0.7, edgecolor='black')
ax.set_xticks(x_pos)
ax.set_xticklabels(['Input'] + [f'H{i+1}' for i in range(len(hidden_dims))] + ['Output'])
ax.set_ylabel('Compression Ratio', fontsize=11)
ax.set_title('Lossy Compression Through Network', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(1, color='black', linestyle='--', alpha=0.5)

# Plot 4: Dimension vs information
ax = axes[1, 1]
ax.scatter(layer_sizes, layer_entropies, s=200, c=range(len(layer_sizes)), 
          cmap='viridis', edgecolor='black', linewidth=2)
for i, (d, e) in enumerate(zip(layer_sizes, layer_entropies)):
    label = 'Input' if i == 0 else f'H{i}' if i <= len(hidden_dims) else 'Output'
    ax.annotate(label, (d, e), xytext=(5, 5), textcoords='offset points', fontsize=10)
ax.set_xlabel('Layer Dimension', fontsize=11)
ax.set_ylabel('Information Content (bits)', fontsize=11)
ax.set_title('Dimension vs Information Content', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔑 Key insights:")
print(f"   1. Input dimension: {input_dim}, Information: {layer_entropies[0]:.2f} bits")
print(f"   2. Bottleneck dimension: {hidden_dims[-1]}, Information: {layer_entropies[len(hidden_dims)]:.2f} bits")
print(f"   3. Information lost: {layer_entropies[0] - layer_entropies[len(hidden_dims)]:.2f} bits ({(1 - layer_entropies[len(hidden_dims)]/layer_entropies[0])*100:.1f}%)")
print(f"   4. This is INTENTIONAL! Network forgets noise, remembers structure.")
print("\n   → Irreversibility (information loss) is essential for learning!")
print("   → Networks that compress well generalize well.")
print("   → Thermodynamic analogy: Compress = coarse-grain = increase entropy")
print("\n✓ Information bottleneck demonstration complete!")

# 第8节：时间箭头 - 基本还是涌现

我们已经从多个角度探讨了不可逆性。现在问一个深刻的问题：

**时间箭头是基本的还是涌现的？**

## 支持"基本"的论点

1. **弱相互作用违反 CP 对称性** → 轻微的时间逆转违反
2. **宇宙学箭头**：宇宙膨胀（不是收缩）
3. **量子测量**：波函数坍缩是不可逆的

## 支持"涌现"的论点

1. **统计力学**：第二定律从统计中涌现
2. **微观可逆性**：基本定律是时间对称的
3. **边界条件**：低熵大爆炸设定了箭头

## 共识：主要是涌现的

时间箭头**不是**在物理定律中——它在**初始条件**中：

$$S_{\text{universe}}(t) > S_{\text{universe}}(0)$$

宇宙始于**极低熵**。从那时起，熵一直在增加。

**为什么大爆炸是低熵的？**我们不知道！这是物理学中最深的谜题之一。

## 意味着：

- **热力学箭头**：熵增加
- **心理学箭头**：我们记住过去，不记住未来（记忆需要低熵）
- **宇宙学箭头**：宇宙膨胀

三者都是**低熵初始条件的后果**。

让我们可视化一下！

In [ ]:
# Visualize the arrow of time

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Entropy vs time for universe
ax = axes[0, 0]
time_universe = np.linspace(0, 13.8, 1000)  # Billion years
# Simplified model: entropy grows logarithmically with time
entropy_universe = np.log(1 + time_universe * 10) + np.random.randn(1000) * 0.1

ax.plot(time_universe, entropy_universe, linewidth=2, color='darkblue')
ax.scatter([0], [entropy_universe[0]], s=200, c='red', marker='*', 
          label='Big Bang (low entropy!)', zorder=5, edgecolor='black', linewidth=2)
ax.arrow(3, 1, 3, 1, head_width=0.2, head_length=0.5, fc='red', ec='red', linewidth=2)
ax.text(4, 2.5, 'Arrow of Time', fontsize=12, color='red', fontweight='bold')
ax.set_xlabel('Time (billion years since Big Bang)', fontsize=11)
ax.set_ylabel('Universe Entropy (arbitrary units)', fontsize=11)
ax.set_title('Cosmological Arrow of Time', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Forward vs backward in time
ax = axes[0, 1]
t = np.linspace(0, 10, 100)
entropy_forward = 1 - np.exp(-t/3)  # Entropy increases
entropy_backward = np.flip(entropy_forward)  # Time-reversed (entropy decreases)

ax.plot(t, entropy_forward, linewidth=3, color='green', label='Forward in time (2nd law holds)')
ax.plot(t, entropy_backward, linewidth=3, color='red', linestyle='--', 
       label='Backward in time (2nd law violated!)')
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('Entropy', fontsize=11)
ax.set_title('Time Asymmetry: Why Backward Looks Wrong', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Phase space volume (Liouville) vs coarse-grained entropy
ax = axes[1, 0]
t = np.linspace(0, 10, 100)
phase_space_volume = np.ones_like(t)  # Constant (Liouville)
macro_entropy = 1 - np.exp(-t/2)  # Increases

ax2 = ax.twinx()
ax.plot(t, phase_space_volume, linewidth=3, color='blue', label='Microscopic (Liouville)')
ax2.plot(t, macro_entropy, linewidth=3, color='red', label='Macroscopic (2nd law)')
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('Phase Space Volume (conserved)', fontsize=11, color='blue')
ax2.set_ylabel('Macroscopic Entropy (increases)', fontsize=11, color='red')
ax.set_title('Microscopic Reversibility vs Macroscopic Irreversibility', 
            fontsize=12, fontweight='bold')
ax.tick_params(axis='y', labelcolor='blue')
ax2.tick_params(axis='y', labelcolor='red')
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='center right')
ax.grid(True, alpha=0.3)

# Plot 4: Three arrows of time
ax = axes[1, 1]
ax.axis('off')

# Draw three arrows
arrow_props = dict(arrowstyle='->', lw=4, color='darkblue')
ax.annotate('', xy=(0.8, 0.8), xytext=(0.2, 0.8), arrowprops=arrow_props)
ax.text(0.5, 0.85, 'Thermodynamic Arrow', ha='center', fontsize=12, fontweight='bold')
ax.text(0.5, 0.75, 'Entropy increases', ha='center', fontsize=10, style='italic')

arrow_props['color'] = 'darkgreen'
ax.annotate('', xy=(0.8, 0.5), xytext=(0.2, 0.5), arrowprops=arrow_props)
ax.text(0.5, 0.55, 'Psychological Arrow', ha='center', fontsize=12, fontweight='bold')
ax.text(0.5, 0.45, 'Remember past, not future', ha='center', fontsize=10, style='italic')

arrow_props['color'] = 'darkred'
ax.annotate('', xy=(0.8, 0.2), xytext=(0.2, 0.2), arrowprops=arrow_props)
ax.text(0.5, 0.25, 'Cosmological Arrow', ha='center', fontsize=12, fontweight='bold')
ax.text(0.5, 0.15, 'Universe expands', ha='center', fontsize=10, style='italic')

ax.text(0.5, 0.05, 'All three aligned → All consequences of low-entropy Big Bang', 
       ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title('The Three Arrows of Time', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔑 Deep insights about time:")
print("\n1. MICROSCOPIC LAWS: Time-reversible (Newtonian, quantum mechanics)")
print("   → Running laws backward is mathematically valid")
print("\n2. MACROSCOPIC BEHAVIOR: Time-irreversible (thermodynamics)")
print("   → Entropy increases, mixing happens, cannot unmix")
print("\n3. THE RESOLUTION: Initial conditions + statistics")
print("   → Big Bang had EXTREMELY low entropy")
print("   → Statistical evolution explores high-entropy states")
print("   → Coarse-graining makes evolution appear irreversible")
print("\n4. THE DEEP MYSTERY: Why was the Big Bang low-entropy?")
print("   → We don't know! This is the origin of the arrow of time.")
print("   → Some theories: anthropic principle, bouncing cosmology, etc.")
print("\n5. THREE ARROWS, ONE CAUSE:")
print("   → Thermodynamic: Entropy ↑ (consequence of initial conditions)")
print("   → Psychological: Memory works backward (requires low entropy)")
print("   → Cosmological: Universe expands (related to initial conditions)")
print("\n✓ Arrow of time analysis complete!")

# 第9节：生物不可逆性 - 生命与第二定律

**生命违反第二定律吗？**

不！生命是一个**开放系统**，它：
1. 局部减少熵（创建秩序）
2. 整体增加熵（输出无序）

## 薛定谔的洞察

在"生命是什么？"（1944）中，薛定谔说：
> "生命以负熵为食"

生物体：
- 输入**低熵**能量（食物、阳光）
- 使用它维持秩序（新陈代谢、生长、繁殖）
- 输出**高熵**废物（热量、CO₂）

**净结果**：宇宙熵增加，第二定律得到满足

## 开放系统的第二定律

对于与环境交换熵 $S_{\text{env}}$ 的开放系统：

$$\frac{dS_{\text{system}}}{dt} + \frac{dS_{\text{env}}}{dt} \geq 0$$

$S_{\text{system}}$ 可以减少，如果 $S_{\text{env}}$ 增加更多！

让我们建模一下！

In [ ]:
# Model a simple "living" system that maintains order

def simulate_open_system(num_steps: int = 200) -> dict:
    """Simulate an open system that maintains low entropy.
    
    Returns:
        Dictionary with entropy histories
    """
    # Initialize
    S_system = 10.0  # System starts with moderate entropy
    S_environment = 0.0  # Track cumulative entropy exported
    
    S_system_history = [S_system]
    S_environment_history = [S_environment]
    S_total_history = [S_system + S_environment]
    
    for t in range(num_steps):
        # System processes: would naturally increase entropy
        natural_increase = 0.5
        
        # Active maintenance: system exports entropy to environment
        # This "costs" more entropy in environment than saved in system
        entropy_exported = 0.8  # More than natural increase (2nd law!)
        entropy_reduced = 0.3  # System entropy reduction
        
        # Update
        S_system = S_system + natural_increase - entropy_reduced
        S_environment = S_environment + entropy_exported
        
        # Record
        S_system_history.append(S_system)
        S_environment_history.append(S_environment)
        S_total_history.append(S_system + S_environment)
    
    return {
        'system': S_system_history,
        'environment': S_environment_history,
        'total': S_total_history
    }


def simulate_closed_system(num_steps: int = 200) -> dict:
    """Simulate a closed system (for comparison)."""
    S_total = 10.0
    S_total_history = [S_total]
    
    for t in range(num_steps):
        # Entropy only increases in closed system
        S_total = S_total + 0.5
        S_total_history.append(S_total)
    
    return {'total': S_total_history}


print("Simulating open system (life-like) vs closed system...\n")

open_sys = simulate_open_system()
closed_sys = simulate_closed_system()

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Open system (life)
ax = axes[0]
t = np.arange(len(open_sys['total']))
ax.plot(t, open_sys['system'], linewidth=2, label='System (organism)', color='green')
ax.plot(t, open_sys['environment'], linewidth=2, label='Environment', color='brown')
ax.plot(t, open_sys['total'], linewidth=3, label='Total (system + env)', 
       color='red', linestyle='--')
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('Entropy', fontsize=11)
ax.set_title('Open System: Life Maintains Order by Exporting Entropy', 
            fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.text(100, 50, 'System entropy\ncan decrease!', fontsize=10, color='green',
       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax.text(100, 120, 'But total entropy\nalways increases!', fontsize=10, color='red',
       bbox=dict(boxstyle='round', facecolor='pink', alpha=0.5))

# Plot 2: Comparison with closed system
ax = axes[1]
ax.plot(t, closed_sys['total'], linewidth=3, label='Closed system', color='blue')
ax.plot(t, open_sys['total'], linewidth=3, label='Open system (total)', 
       color='red', linestyle='--')
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('Total Entropy', fontsize=11)
ax.set_title('Both Systems Obey 2nd Law: Total Entropy ↑', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.text(100, 50, 'Open system increases\nentropy FASTER\n(due to metabolism)', 
       fontsize=10, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n🔑 Key insights about life and entropy:")
print("\n1. LOCAL vs GLOBAL:")
print(f"   → System entropy: {open_sys['system'][0]:.1f} → {open_sys['system'][-1]:.1f} (maintained low!)")
print(f"   → Total entropy:  {open_sys['total'][0]:.1f} → {open_sys['total'][-1]:.1f} (increased!)")
print("\n2. LIFE'S TRICK:")
print("   → Import low-entropy energy (food, sunlight)")
print("   → Use it to build order (proteins, cells, organisms)")
print("   → Export high-entropy waste (heat, CO₂, etc.)")
print("\n3. NET RESULT:")
print("   → Organism entropy ↓ (more order)")
print("   → Environment entropy ↑↑ (much more disorder)")
print("   → Total entropy ↑ (2nd law satisfied!)")
print("\n4. WHY IT WORKS:")
print("   → Exporting entropy is IRREVERSIBLE")
print("   → Heat cannot spontaneously reconcentrate")
print("   → This is why death is inevitable (eventually can't export enough)")
print("\n5. SCHRÖDINGER WAS RIGHT:")
print("   → Life feeds on negative entropy (order)")
print("   → Exports positive entropy (disorder)")
print("   → This is how life is compatible with 2nd law!")
print("\n✓ Biological irreversibility demonstration complete!")

# 第10节：综合 - 各尺度的不可逆性

我们已经从咖啡杯到宇宙探讨了不可逆性。让我们综合一下。

## 不可逆性的层次

| 尺度 | 系统 | 可逆？ | 为什么不可逆？ |
|-------|--------|-------------|-------------------|
| **微观** | 单个粒子 | ✅ 是 | 牛顿/量子定律是时间对称的 |
| **介观** | 小群体（~100s） | ⚠️ 实际不 | 庞加莱回归时间 >> 观察时间 |
| **宏观** | 日常物体 | ❌ 否 | 粗粒化 + 统计学 + 高 N |
| **宇宙学** | 宇宙 | ❌ 否 | 低熵初始条件 |

## 共同主线

所有形式的不可逆性都分享：

1. **粗粒化**：丢失细粒度信息
   - 物理学：宏观态 vs 微观态
   - 计算：哈希函数，有损压缩
   - ML：神经网络瓶颈

2. **多对一映射**：多个输入 → 相同输出
   - 咖啡：许多分子配置 → 相同宏观外观
   - 哈希：许多字符串 → 相同哈希值
   - 神经网络：许多图像 → 相同类别

3. **信息丢失**：无法恢复原始状态
   - 热力学：熵增加
   - 兰道尔：位擦除成本能量
   - ML：压缩丢弃细节

4. **统计偏差**：向高概率状态演化
   - 物理学：平衡态最可能
   - 计算：状态空间中的随机游走
   - ML：梯度下降趋向极小值

## 深刻含义

### 对于物理学
- 时间是涌现的，不是基本的
- 第二定律是统计的，不是绝对的
- 低熵初始条件是关键谜题

### 对于计算
- 所有计算都有热力学成本
- 可逆计算可能"免费"
- 信息是物理的（不是抽象的）

### 对于机器学习
- 压缩是本质的（用于泛化）
- 信息瓶颈 = 好的归纳偏差
- 遗忘（不可逆）有助于学习

### 对于生命
- 开放系统可以维持秩序
- 但必须输出熵
- 死亡是热力学必然性

## 终极问题

**为什么宇宙诞生时熵如此低？**

这单一事实解释了：
- 时间箭头
- 为什么混合是不可逆的
- 为什么我们记住过去，不记住未来
- 为什么生命可能存在
- 为什么计算是可能的

我们不知道答案。但这是物理学中最深的问题之一。

---

## 与机器学习的联系（重新审视）

**为什么这对 AI 很重要？**

1. **信息瓶颈**：神经网络必须压缩 → 不可逆
2. **单向函数**：安全性取决于计算不可逆性
3. **兰道尔限制**：未来 AI 能效受热力学限制
4. **记忆**：大脑/计算机必须擦除旧记忆 → 熵成本
5. **学习 = 压缩**：好的模型压缩数据不可逆地

**不可逆性不是 bug——它是特性！**

没有它：
- 没有泛化（会记住一切）
- 没有计算（没有单向函数）
- 没有安全性（可以逆转任何函数）
- 没有学习（没有抽象）

**你不能取消咖啡的混合。但如果你能，你无法思考它。**

In [ ]:
# Final visualization: The complete picture

fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Central concept: The arrow of time
ax_center = fig.add_subplot(gs[1, 1])
ax_center.axis('off')
ax_center.text(0.5, 0.5, '☕\n\nThe Coffee Automaton\n\nIrreversibility', 
              ha='center', va='center', fontsize=20, fontweight='bold',
              bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
ax_center.set_xlim(0, 1)
ax_center.set_ylim(0, 1)

# Surrounding concepts
concepts = [
    ("Physics\n\nEntropy ↑\n2nd Law", gs[0, 0]),
    ("Phase Space\n\nLiouville\nCoarse-graining", gs[0, 1]),
    ("Computation\n\nOne-way\nLandauer", gs[0, 2]),
    ("Recurrence\n\nPoincaré\ne^N time", gs[1, 0]),
    ("Maxwell\n\nDemon\nInfo=Physical", gs[1, 2]),
    ("Biology\n\nLife\nOpen system", gs[2, 0]),
    ("ML\n\nCompression\nBottleneck", gs[2, 1]),
    ("Cosmology\n\nBig Bang\nLow entropy", gs[2, 2]),
]

colors = ['lightcoral', 'lightgreen', 'lightyellow', 'lightpink', 
         'lightcyan', 'lavender', 'peachpuff', 'thistle']

for (text, pos), color in zip(concepts, colors):
    ax = fig.add_subplot(pos)
    ax.axis('off')
    ax.text(0.5, 0.5, text, ha='center', va='center', fontsize=12,
           bbox=dict(boxstyle='round', facecolor=color, alpha=0.7))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    # Draw arrow to center
    ax.annotate('', xy=(0.5, 0.5), xytext=(0.5, 0.5),
               arrowprops=dict(arrowstyle='->', lw=2, color='gray', alpha=0.5))

plt.suptitle('Irreversibility: A Unifying Concept Across All Scales', 
            fontsize=16, fontweight='bold', y=0.98)
plt.show()

# Summary statistics
print("\n" + "="*70)
print("SUMMARY: THE COFFEE AUTOMATON - KEY TAKEAWAYS")
print("="*70)

print("\n1. FUNDAMENTAL PUZZLE:")
print("   • Microscopic laws are reversible (Newton, Schrödinger)")
print("   • Macroscopic behavior is irreversible (coffee mixes, never unmixes)")
print("   • Resolution: Coarse-graining + statistics + low-entropy initial condition")

print("\n2. MECHANISMS OF IRREVERSIBILITY:")
print("   • Coarse-graining: Losing information when grouping microstates")
print("   • Statistical mechanics: High-entropy states vastly outnumber low-entropy")
print("   • Poincaré recurrence: Reversible, but on timescale e^N >> universe age")

print("\n3. INFORMATION IS PHYSICAL:")
print("   • Landauer's principle: Erasing 1 bit costs k_B T ln(2) energy")
print("   • Maxwell's demon: Information gathering/erasure has entropic cost")
print("   • Computation: All irreversible operations dissipate heat")

print("\n4. COMPUTATIONAL IRREVERSIBILITY:")
print("   • One-way functions: Easy forward, hard backward")
print("   • Cryptographic hashing: Many inputs → same output")
print("   • Security depends on computational irreversibility")

print("\n5. MACHINE LEARNING:")
print("   • Neural networks compress (information bottleneck)")
print("   • Compression = irreversible = forgetting details")
print("   • Generalization requires irreversibility!")

print("\n6. LIFE AND THERMODYNAMICS:")
print("   • Life is open system: imports order, exports disorder")
print("   • Local entropy ↓, but total entropy ↑ (2nd law satisfied)")
print("   • Death is thermodynamic inevitability")

print("\n7. THE ARROW OF TIME:")
print("   • Not in the laws—in the initial conditions!")
print("   • Big Bang had extremely low entropy (why??)")
print("   • All three arrows (thermodynamic, psychological, cosmological) aligned")

print("\n8. DEEP MYSTERY:")
print("   • Why was the Big Bang low-entropy?")
print("   • This single fact explains the arrow of time")
print("   • Still unsolved! One of deepest questions in physics")

print("\n" + "="*70)
print("\n☕ The coffee automaton: Simple system, profound implications")
print("\nYou can't unmix the coffee.")
print("But this irreversibility is what makes computation, life, and thought possible.")
print("\n" + "="*70)
print("\n✓ Complete analysis of irreversibility finished!")
print("\n🎓 Paper 19 implementation complete: A deep dive into the coffee automaton.")